In [132]:
import numpy as np
import pandas as pd
import re
import json

In [2]:
df = pd.read_csv("crimes.csv")

In [48]:
len(df)

4436

In [27]:
#latitude: x, longitude:y
def get_lat(string):
    if len(re.findall(r"\(.*\)", string)) == 0:
        return 0.0
    return float(re.findall(r"\(.*\)", string)[0].replace('(', '').replace(')', '').replace(' ', '').split(',')[0])

In [28]:
def get_lon(string):
    if len(re.findall(r"\(.*\)", string)) == 0:
        return 0.0
    return float(re.findall(r"\(.*\)", string)[0].replace('(', '').replace(')', '').replace(' ', '').split(',')[1])

In [113]:
df['latitude'] = df['Block_Location'].apply(get_lat)
df['longitude'] = df['Block_Location'].apply(get_lon)

In [114]:
df['latitude'].mean()

36.51957090238948

In [115]:
df['longitude'].mean()

-117.9214566882329

In [116]:
df[df['Latitude'] == 0.0]['Latitude'] = df['Latitude'].mean()

/Users/sethdumaguin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [117]:
df[df['Longitude'] == 0.0]['Longitude'] = df['Longitude'].mean()

/Users/sethdumaguin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [118]:
def get_crime(crime):
    return crime.replace("/", " ").split(' ')[0]

In [119]:
df['Short Crime'] = df['OFFENSE'].apply(get_crime)

In [120]:
crime_score = pd.read_csv('crime_score.csv')

In [121]:
crime_score.head()

,Crime,Score
0,RECOVERED VEHICLE,0
1,VANDALISM,1
2,NOISE VIOLATION,2
3,DRUG VIOLATION,3
4,LIQUOR LAW VIOLATION,3


In [123]:
crime_score = {
    "RECOVERED VEHICLE" : 0,
    "VANDALISM": 1,
    "NOISE VIOLATION" : 2, 
    "DRUG VIOLATION" : 3, 
    "LIQUOR LAW VIOLATION": 3, 
    "DISORDERLY CONDUCT": 4, 
    "FAMILY OFFENSE": 5, 
    "WEAPONS OFFENSE" : 5, 
    "BURGLARY - RESIDENTIAL" : 6, 
    "BURGLARY - VEHICLE" : 6, 
    "FRAUD": 6, 
    "BURGLARY - COMMERCIAL" : 6, 
    "ROBBERY" : 6, 
    "MOTOR VEHICLE THEFT": 6, 
    "LARCENY": 7,
    "LARCENY - FROM VEHICLE": 7, 
    "ASSAULT": 8, 
    "SEX CRIME": 9, 
    "MISSING PERSON" : 10, 
    "KIDNAPPING" : 11, 
    "ARSON" :12
}

In [124]:
scores = []
for i in df["CVLEGEND"]:
    scores.append(crime_score[i])

df['Score'] = scores

In [125]:
df.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Latitude,Longitude,Short Crime,Score,latitude,longitude
0,18063668,BURGLARY RESIDENTIAL,11/14/2018 12:00:00 AM,02:48,BURGLARY - RESIDENTIAL,3,04/14/2019 07:00:12 AM,"1000 PARDEE ST\nBerkeley, CA\n(37.856698, -122...",1000 PARDEE ST,Berkeley,CA,37.856698,-122.290343,BURGLARY,6,37.856698,-122.290343
1,18062188,NARCOTICS,11/05/2018 12:00:00 AM,14:27,DRUG VIOLATION,1,04/14/2019 07:00:12 AM,"2300 BANCROFT WAY\nBerkeley, CA\n(37.868199, -...",2300 BANCROFT WAY,Berkeley,CA,37.868199,-122.263253,NARCOTICS,3,37.868199,-122.263253
2,19090551,BURGLARY AUTO,03/07/2019 12:00:00 AM,10:15,BURGLARY - VEHICLE,4,04/14/2019 07:00:18 AM,"800 BOLIVAR DR\nBerkeley, CA",800 BOLIVAR DR,Berkeley,CA,0.000000,0.000000,BURGLARY,6,0.000000,0.000000
3,18092014,VANDALISM,11/14/2018 12:00:00 AM,17:00,VANDALISM,3,04/14/2019 07:00:12 AM,"VINE STREET &amp; MILVIA ST\nBerkeley, CA\n(37...",VINE STREET & MILVIA ST,Berkeley,CA,37.879921,-122.271646,VANDALISM,1,37.879921,-122.271646
4,19016735,THEFT FELONY (OVER $950),03/28/2019 12:00:00 AM,23:00,LARCENY,4,04/14/2019 07:00:12 AM,"1600 FIFTH ST\nBerkeley, CA\n(37.873865, -122....",1600 FIFTH ST,Berkeley,CA,37.873865,-122.300777,THEFT,7,37.873865,-122.300777


In [126]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [134]:
cols = ['BLKADDR', 'Score', 'CVLEGEND']
geojson = df_to_geojson(df, cols)
geojson_str = json.dumps(geojson, indent=2)

In [135]:
output_filename = 'dataset.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))